In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import datetime
import numpy as np
from matplotlib import pyplot as plt
import os

import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats as st

In [6]:
price_data = pd.read_csv('datasets/btc-usdt-1.csv')

In [234]:
def getTrainingDataset(ds, num_lags, prediction_interval = 30):


    ## generate some features so that our model could learn something
    
    out_ds = pd.DataFrame()
    out_ds['log_return_ask'] = np.log(ds['ask_price'].pct_change() + 1)
    out_ds['log_return_bid'] = np.log(ds['bid_price'].pct_change() + 1)
    
    out_ds['log_ask_div_bid'] = np.log(ds['ask_price'] / ds['bid_price'])
    out_ds['log_volume_ask_div_bid'] = np.log(ds['ask_size'] / ds['bid_size'])
    
    out_ds['log_volume_ask'] = np.log(ds['ask_size'])
    out_ds['log_volume_bid'] = np.log(ds['bid_size'])

            
    mid_price = (ds['ask_price'] + ds['bid_price']) / 2
    out_ds['log_return_mid_price'] = np.log(mid_price.pct_change() + 1)
    
    cols_features = out_ds.columns
    out_ds = out_ds.assign(**{
        '{}_(t-{})'.format(col, t): out_ds[col].shift(t)
        for t in list(range(1, num_lags))
        for col in cols_features})

    targets = np.log(mid_price.shift(-prediction_interval)/mid_price)
    out_ds['y'] = targets
    return out_ds

In [235]:
training_dataset = getTrainingDataset(price_data, 10)

training_dataset = training_dataset.dropna()
features_columns = training_dataset.columns[training_dataset.columns != 'y']
target_column = 'y'

training_dataset.head(2)

,log_return_ask,log_return_bid,log_ask_div_bid,log_volume_ask_div_bid,log_volume_ask,log_volume_bid,log_return_mid_price,f1,log_return_ask_(t-1),log_return_bid_(t-1),...,f1_(t-3),log_return_ask_(t-4),log_return_bid_(t-4),log_ask_div_bid_(t-4),log_volume_ask_div_bid_(t-4),log_volume_ask_(t-4),log_volume_bid_(t-4),log_return_mid_price_(t-4),f1_(t-4),y
our_time,,,,,,,,,,,,,,,,,,,,,
2022-09-21 03:00:05,-0.000027,-0.000007,0.000033,-2.335369,-2.135716,0.199654,-0.000017,-0.001832,-0.000022,0.000009,...,0.000264,-0.000081,-0.000076,0.000031,-4.605170,-11.512925,-6.907755,-0.000078,0.000064,0.000047
2022-09-21 03:00:06,0.000050,0.000045,0.000038,0.806436,-4.606171,-5.412607,0.000047,-0.000014,-0.000027,-0.000007,...,0.000684,-0.000045,-0.000047,0.000033,-0.135856,-3.354982,-3.219126,-0.000046,0.000264,-0.000010


In [236]:
import numpy as np
from sklearn.model_selection import train_test_split
def get_train_val_test_split(training_dataset, features_columns = features_columns, target_column = target_column, test_ratio = 0.25):
    val_ratio = test_ratio / (1 - test_ratio)
    X_train, X_test, y_train, y_test = train_test_split(
        training_dataset[features_columns],
        training_dataset[target_column],
        test_size=test_ratio, 
        shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio, shuffle=False)
    return X_train, X_val, X_test, y_train, y_val, y_test